# Homework 1 (2022) : Data Wrangling and Visualization

Due date : **2022-01-30 @23h55** (this is a **hard deadline**)

## Fill this cell with your names

- Name, First Name, Informatique/Mathématique-Informatique
- Name, First Name, Informatique/Mathématique-Informatique

## Carefully follow instructions given on the [course homepage](https://stephanegaiffas.github.io/big_data_course/homeworks/)

Write in English or French

- The deliverable is a `xxx.ipynb` file (jupyter notebook) or a `xxx.py` file (if you are using `jupytext`) built by completing the template. **We won't execute the code in your notebook:** all your results, displays and plots must be visible without having to rerun everything.

Once again, follow the steps described here : [course homepage](https://stephanegaiffas.github.io/big_data_course/homeworks/). **If you don't: no evaluation!**

## Grading <i class="fa graduation-cap"></i>

Here is the way we'll assess your work

| Criterion | Points | Details |
|:----------|:-------:|:----|
|Spelling and syntax | 3 | English/French  |
|Plots correction | 3 |  Clarity / answers the question  |
|Plot style and cleanliness | 3 | Titles, legends, labels, breaks ... |
|Table wrangling | 4 | ETL, SQL like manipulations |
|Computing Statistics | 5 | SQL `goup by`  and aggregation  |
|DRY compliance | 2 | DRY principle at [Wikipedia](https://en.wikipedia.org/wiki/Don%27t_repeat_yourself)|

If we see a single (or more) `for` loop in your code: **-5 points**.  Everything can be done using high-level `pandas` methods

# Preliminaries

## Notebook: Modus operandi

- This is a [Jupyter Notebook]().
- When you execute code within the notebook, the results appear beneath the code.
- [Jupytext]()

## Packages

- Base `Python` can do a lot. But the full power of `Python` comes from a fast growing collection of `packages`/`modules`.

- Packages are first installed (that is using `pip install` or `conda install`), and if
needed, imported during a session.

- The `docker` image you are supposed to use already offers a lot of packages. You should not need to install new packages.

- Once a package has been installed on your drive, if you want all objects exported by the package to be available in your session, you should import the package, using `from pkg import *`.

- If you just want to pick some sobjects from the package,
you can use qualified names like `pkg.object_name` to access the object (function, dataset, class...)


In [1]:
# importing basic tools
import numpy as np
import pandas as pd

from pandas.api.types import CategoricalDtype

import os            # file operations
import requests      # networking

from datetime import date  # if needed

In [2]:
# importing plotting packages
import plotly.graph_objects as go
import plotly.express as px
import plotly.io as pio

In [3]:
# make pandas plotly-friendly
np.set_printoptions(precision=2, suppress=True)
%matplotlib inline
pd.options.plotting.backend = "plotly"

# Getting the data

The data are built and made available by [INSEE](https://www.insee.fr/fr/accueil)  (French Governement Statistics Institute)

Prénoms:
- [https://www.insee.fr/fr/statistiques/fichier/2540004/nat2020_csv.zip](https://www.insee.fr/fr/statistiques/fichier/2540004/nat2020_csv.zip)

This dataset has been growing for a while. It has been considered by
social scientists for decades.  Given names are meant to give insights into a variety
of phenomena, including religious observance.

- A glimpse at the body of work can be found in [_L'archipel français_ by Jérome Fourquet, Le Seuil, 2019 ](https://www.seuil.com/ouvrage/l-archipel-francais-jerome-fourquet/9782021406023)

- Read the [File documentation](https://www.insee.fr/fr/statistiques/2540004?sommaire=4767262#documentation)

## Download the data

**QUESTION:** Download the data into a file which relative path is `'./nat2020_csv.zip'`

__Hints:__

- Have a look at [`requests`](https://requests.readthedocs.io/en/master/).
- Use magic commands to navigate across the file hierarchy and create subdirectories
when needed

In [4]:
params = dict(
    url = 'https://www.insee.fr/fr/statistiques/fichier/2540004/nat2020_csv.zip',
    dirpath = './',
    timecourse = '',
    datafile = 'nat2020.hdf',
    fpath = 'nat2020_csv.zip'
)

In [5]:
# modify location  make sure you are in the right directory
# %cd
!ls -alh
# %pwd  #
# %ls
# %mkdir # if needed

total 7.9M
drwxrwxr-x 5 1000 1000 4.0K Jan 29 02:09 .
drwxr-xr-x 1 1001  115 4.0K Jan 14  2021 ..
drwx------ 4 root root 4.0K Jan 28 20:23 .Trash-0
drwxr-xr-x 2 root root 4.0K Jan 28 20:23 .ipynb_checkpoints
drwxrwxr-x 3 1000 1000 4.0K Jan 28 20:21 Big-Data-Technologies
-rw-rw-r-- 1 1000 1000  370 Jan 22 02:32 docker-compose.yml
-rw-r--r-- 1 root root  163 Jan 26 09:00 fruits.csv
-rw-r--r-- 1 root root 132K Jan 26 09:41 gro.csv.gz
-rw-r--r-- 1 root root 830K Jan 26 09:41 gro_cleaned.csv
-rw-r--r-- 1 root root 249K Jan 26 09:41 gro_cleaned.feather
-rw-r--r-- 1 root root 203K Jan 26 09:41 gro_cleaned.parquet
-rw-r--r-- 1 root root 633K Jan 26 09:41 gro_cleaned.pkl
-rw-r--r-- 1 root root 737K Jan 26 09:41 gro_training.pkl
-rw-r--r-- 1 root root  66K Jan 29 02:09 homework01.ipynb
-rw-r--r-- 1 root root 2.4M Jan 28 20:26 nat2020_csv.zip
-rw-r--r-- 1 root root 196K Jan 19 10:51 notebook01_python.ipynb
-rw-r--r-- 1 root root 1.7M Jan 25 23:11 notebook02_numpy.ipynb
-rw-r--r-- 1 root root 329K

In [6]:
url = params['url']      # 'https://www.insee.fr/fr/statistiques/fichier/2540004/nat2020_csv.zip'
fpath = params['fpath']  # './nat2020_csv.zip'

In [7]:
# your code here
dirpath = params['dirpath']
if os.path.exists(os.path.join(dirpath, fpath)):
    print('Le fichier %s exsite déjà.' % os.path.join(dirpath, fpath))
else:
    r = requests.get(url)
    with open(os.path.join(dirpath, fpath), 'wb') as f:
        f.write(r.content)
    print('Télécharger le fichier %s.' % os.path.join(dirpath, fpath))


Le fichier ./nat2020_csv.zip exsite déjà.


## Load the data in memory

**QUESTION:** Load the data in a `pandas` `DataFrame` called `data`

__Hints:__

- You should obtain a `Pandas dataframe` with 4 columns.
- Mind the conventions used to build theis `csv` file.
- Package `pandas` provides the convenient tools.
- The dataset, though not too large, is already demanding.
- Don't hesitate to test your methods on a sample of rows method `sample()` from class `DataFrame` can be helpful.

In [8]:
# your code here
data = pd.read_csv('nat2020_csv.zip', compression='zip', sep=';')
data.sample(10)

,sexe,preusuel,annais,nombre
386330,2,DJINA,1974,3
152517,1,JUAN,2017,90
113308,1,HAMIDE,XXXX,38
313280,2,ALESIA,1998,9
612234,2,SALIA,2005,10
30418,1,ASKIN,XXXX,19
16649,1,ALKAN,2007,4
363631,2,CHAHINAZE,1991,8
457644,2,JEANNETTE,1942,371
588202,2,OXANE,2007,11


In [9]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 667364 entries, 0 to 667363
Data columns (total 4 columns):
 #   Column    Non-Null Count   Dtype 
---  ------    --------------   ----- 
 0   sexe      667364 non-null  int64 
 1   preusuel  667362 non-null  object
 2   annais    667364 non-null  object
 3   nombre    667364 non-null  int64 
dtypes: int64(2), object(2)
memory usage: 20.4+ MB


## Explore the data

In [10]:
data.columns

Index(['sexe', 'preusuel', 'annais', 'nombre'], dtype='object')

In [11]:
data.dtypes

sexe         int64
preusuel    object
annais      object
nombre       int64
dtype: object

In [12]:
data.head()

,sexe,preusuel,annais,nombre
0,1,_PRENOMS_RARES,1900,1250
1,1,_PRENOMS_RARES,1901,1342
2,1,_PRENOMS_RARES,1902,1330
3,1,_PRENOMS_RARES,1903,1286
4,1,_PRENOMS_RARES,1904,1430


In [13]:
data.describe(include="all")

,sexe,preusuel,annais,nombre
count,667364.000000,667362,667364,667364.000000
unique,NaN,35010,122,NaN
top,NaN,_PRENOMS_RARES,XXXX,NaN
freq,NaN,242,36675,NaN
mean,1.543828,NaN,NaN,129.772665
std,0.498076,NaN,NaN,884.912504
min,1.000000,NaN,NaN,1.000000
25%,1.000000,NaN,NaN,4.000000
50%,2.000000,NaN,NaN,8.000000
75%,2.000000,NaN,NaN,25.000000


**QUESTION:** Look at the data, Use the attributes `columns`, `dtypes` and the methods `head`, `describe`, to get a feeling of the data.

- This dataset is supposed to report all given names used
for either sex during a year in France since 1900

- The file is made of `652 056` lines and  4 columns.

```
|-- preusuel : object
|-- nombre: int64
|-- sexe: int64
|-- annais: object
```

Each row indicates for a given `preusuel` (prénom usuel, given name), `sexe` (sex), and `annais` (année naissance, birthyear) the `nombre` (number) of babies of the given sex who were given that name during the given year.

|sexe    |preusuel     | annais|   nombre|
|:------|:--------|----:|---:|
|2     |SYLVETTE | 1953| 577|
|1   |BOUBOU   | 1979|   4|
|1   |NILS     | 1959|   3|
|2   |NICOLE   | 2003|  36|
|1   |JOSÉLITO | 2013|   4|


**QUESTION:** Compare memory usage and disk space used by data

**Hints:**

- The method `info`  prints a concise summary of a `DataFrame`.
- With optional parameter `memory_usage`, you can get an estimate
of the amount of memory used by the `DataFrame`.
- Beware that the resulting estimate depends on the argument fed.

In [14]:
# your code here
data.info(memory_usage='deep')
#data.memory_usage()

print("disk space used : ",round(os.stat(fpath).st_size/(1024**2),3),"MB")

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 667364 entries, 0 to 667363
Data columns (total 4 columns):
 #   Column    Non-Null Count   Dtype 
---  ------    --------------   ----- 
 0   sexe      667364 non-null  int64 
 1   preusuel  667362 non-null  object
 2   annais    667364 non-null  object
 3   nombre    667364 non-null  int64 
dtypes: int64(2), object(2)
memory usage: 90.7 MB
disk space used :  2.331 MB


In [15]:
#Disk space used = 20.4+ MB
#Memory usage = 90.7 MB

**QUESTION:** Display the output of `.describe()` with style.

In [16]:
# your code here
def color_nan_red(val):
    color = 'red' if val is np.nan else 'black'
    return 'color: %s' % color

data.describe(include='all').style.\
    applymap(color_nan_red)
#    applymap(lambda x:"%.2f" % x,subset=['mean','std','min','max'])
#data.describe(include='all')

,sexe,preusuel,annais,nombre
count,667364.000000,667362,667364,667364.000000
unique,nan,35010,122,nan
top,nan,_PRENOMS_RARES,XXXX,nan
freq,nan,242,36675,nan
mean,1.543828,nan,nan,129.772665
std,0.498076,nan,nan,884.912504
min,1.000000,nan,nan,1.000000
25%,1.000000,nan,nan,4.000000
50%,2.000000,nan,nan,8.000000
75%,2.000000,nan,nan,25.000000


**QUESTION:** For each column compute the number of distinct values

In [17]:
# your code here
data.nunique()

sexe            2
preusuel    35010
annais        122
nombre       7304
dtype: int64

# Transformations

## Improving the data types

**QUESTION:** Make `sexe` a category with two levels `Female` and `Male`. Call the new column `genre`. Do you see any reason why this factor should be ordered?

__Hint:__ Read [Pandas and categorical variables](https://pandas.pydata.org/pandas-docs/stable/user_guide/categorical.html?highlight=category)

In [18]:
# your code here
data['genre']=data['sexe'].astype('category')
data.genre=data.genre.cat.rename_categories(['Male','Female'])
data.sample(10)

,sexe,preusuel,annais,nombre,genre
466561,2,JUNA,2004,8,Female
142764,1,JEANNICK,1972,14,Male
77519,1,ELRIC,1985,3,Male
555824,2,MICHELINE,1928,4133,Female
257128,1,SERGIO,2008,31,Male
627531,2,SIMONE,1940,3287,Female
14511,1,ALESSANDRO,1977,18,Male
195085,1,MAURICE,1932,7407,Male
368676,2,CHRISTIANE,1926,3025,Female
658500,2,YAMNA,1962,9,Female


**QUESTION:** Compare memory usage of columns `sexe` and `genre`

In [19]:
# your code here
print("memory usage of sexe : ",data.sexe.memory_usage())
print("memory usage of genre : ",data.genre.memory_usage())

memory usage of sexe :  5339040
memory usage of genre :  667616


**QUESTION:** Would it be more memory-efficient to recode `sexe` using modalities `F` and `M` instead of `Male` and `Female` ?

_Insert your answer here_

> Non, comme F et Female sont du même type, la mémoire utilisée est la même quelle que soit celle utilisée

## Dealing with missing values

**QUESTION:** Variable `annais` class is `object`. Make `annais` of type `float`. Note that missing years are encoded as "XXXX", find a way to deal with that.

__Hint:__  As of releasing this Homework (2021-01-22), `Pandas` is not very good at managing missing values,
see [roadmap](https://pandas.pydata.org/docs/development/roadmap.html). Don't try to convert `annais` into an integer column.

In [20]:
# your code here
data = data.replace({'annais' : 'XXXX'},np.nan)
data['annais']=data['annais'].astype('float')

data.sample(10)

,sexe,preusuel,annais,nombre,genre
303033,1,ZEYNEL,2013.0,5,Male
235398,1,RAEL,2003.0,3,Male
660679,2,YOLA,2001.0,6,Female
364214,2,CHAINESE,1998.0,8,Female
263323,1,SOHAIL,2020.0,9,Male
361528,2,CELESTE,1992.0,32,Female
288204,1,WACLAW,1927.0,12,Male
273039,1,TED,2007.0,8,Male
461762,2,JOHANIE,1985.0,8,Female
213776,1,NAYL,2017.0,15,Male


## Rename and remove columns

**QUESTION:** Remove useless columns (now that you've created new ones, and rename them). You should end up with a dataframe with columns called `"gender"`, `"year"`, `"count"`, `"firstname`" with the following dtypes:

```python
gender        category
firstname     object
count         int64
year          float64
```

In [21]:
# your code here
data = data.drop(['sexe'], axis=1)
data = data.rename(columns={'genre':'gender','annais' : 'year','nombre' : 'count','preusuel' : 'firstname'})
data.dtypes

firstname      object
year          float64
count           int64
gender       category
dtype: object

**QUESTION:** How many missing values (NA) have been introduced? How many births are concerned?

In [41]:
# your code here
print("Number of missing values : ",data.isnull().sum().sum())
print("Number of missing birth's values : ",data['year'].isnull().sum())

Number of missing values :  36677
Number of missing birth's values :  36675


**QUESTION:** Read the documentation and describe the origin of rows containing the missing values.

# your code here
Les valeurs manquantes dans la colonne "year" sont des prénoms qui a été attribué au moins 20 fois à des personnes de sexe féminin/masculin sur la période allant de 1946 à 2020, mais il existe des année qui a été attribué moins de 3 fois.

## Checkpointing: save your transformed dataframe

In [24]:
# your code here


**QUESTION:** Save the transformed dataframe (retyped and renamed) to `./nat2020.hdf` using `.hdf` format

In [25]:
# your code here


At that point your working directory should look like:

```
├── homework01.py      # if you use `jupytext`
├── homework01.ipynb
├── nat2020.hdf
├── nat2020.zip
```

**QUESTION:** Reload the data using `read_hdf(...)` so that the resulting dataframe  is properly typed with meaningful column names.

__Hint:__ use `try: ... except` to handle exceptions such as `FileNotFoundError`

In [26]:
# your code here


## Some data "analytics" and visualization

**QUESTION**: For each year, compute the total number of Female and Male births and the proportion of Female  births among total births

__Hints:__

- Groupby operations using several columns for the groups return a dataframe with a `MultiIndex` index see [Pandas advanced](https://pandas.pydata.org/pandas-docs/stable/user_guide/advanced.html)

- Have a look at `MultiIndex`, `reset_index`, `pivot`, `columns.droplevel`

In [70]:
# your code here
births = data.groupby(['year','gender']).sum()\
    .reset_index()\
    .pivot(index='year',columns='gender',values='count')

def propor(a,b):
    return a/(a+b)

#births['proportion'] = births.apply(lambda row: propor(row['Female'],row['Male']), axis=1)

births

gender,Male,Female
year,,
1900.0,177388,237653
1901.0,195964,257492
1902.0,204353,261437
1903.0,207358,261450
1904.0,214140,264822
...,...,...
2016.0,389909,370593
2017.0,382593,363885
2018.0,377802,358460


**QUESTION:** Plot the proportion of female births as a function of year. Don't forget: title, axes labels, ticks, scales, etc.

Because of what we did before, the `plot` method of a `DataFrame` with be rendered using `plotly`, so you can use this. But you can use also `seaborn` or any other available plotting library that you want.

__Hint:__ Mind the missing values in the `year` column

In [28]:
# your code here


**QUESTION:** Make any sensible comment about this plot.

_Insert your answer here_

> ...

**QUESTION:** Explore the fluctuations of sex ratio around its mean value since 1945.
Plot deviations of sex ratio around its mean since 1945 as a function of time.

In [29]:
# your code here


# Popular firstnames

## The top-20 most popular firstnames since 1900

**QUESTION:** For each firstname and sex (some names may be given to girls and boys), compute the total number of times this firstname has been given during `1900-2019`. Print the top 20 firstnames given and style your result dataframe using `background_gradient` for instance.

In [30]:
# your code here


## Rare firstnames

**QUESTION:** For each sex, plot the proportion of births given `_PRENOMS_RARES` as a function of the year.

In [31]:
# your code here


# A study of the "Marie" firstname

**QUESTION:** Plot the proportion of female births given name `'MARIE'` or `'MARIE-...'` (compounded names) as a function of the year.
Proceed in such a way that the reader can see the share of compounded names. We are expecting an _area plot_.

__Hints:__

- Have a look at the `.str` accessor (to apply a string method over a whole column containing string)
- Have a look at [r-graph-gallery: stacked area](https://www.r-graph-gallery.com/stacked-area-graph.html)  and
at [ggplot documentation](https://ggplot2.tidyverse.org/reference/geom_ribbon.html). Pay attention on the way you stack the area corresponding to names matching pattern 'MARIE-.*' over or under the are corresponding to babies named 'MARIE'
- See Graphique 3, page 48, de _L'archipel français_  de J. Fourquet. Le Seuil. Essais. Vol. 898.

- Add annotation, 1st World War, Front Populaire, 2nd World War, 1968

In [32]:
# your code here


# Top 10 firstnames of year 2000

**QUESTION:** For each sex, select the ten most popular names in year 2000, and plot the proportion
of newborns given that name over time. Take into account that some names might have
zero occurrence during certain years.

__Hint:__ Leave aside the rows with '_PRENOMS_RARES'.

In [33]:
# your code here


# Getting help

- [pandas](https://pandas.pydata.org/pandas-docs/stable/reference/)

- [plotly](https://plotly.com/python/) for animated plots

- [stackoverflow](https://stackoverflow.com)

- [stackoverflow: pandas](https://stackoverflow.com/questions/tagged/pandas)

- [stackoverflow: plotly+python](https://stackoverflow.com/questions/tagged/plotly+python)

- The US `babynames` analogue of the INSEE file has been a playground for data scientists,
 see [https://github.com/hadley/babynames](https://github.com/hadley/babynames)

- Don't Repeat Yourself (DRY) principle  at [Wikipedia](https://en.wikipedia.org/wiki/Don%27t_repeat_yourself)